# Generate daily bars using daily data service - F#

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot *`daily bars`*.<br>

It enables to retrieve **on-demand** daily bars data by calling a dedicated service on a **daily** data store. 

### Inputs/outputs
Bars sample requires instrument's identifier as per input. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyBarsService*: to directly retrieve daily bars objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily bars sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.38.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Daily.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily bars* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyBarsService* in order to retrieve daily data

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily bars service
let service = new DailyBarsService.DailyBarsServiceClient(channel)

// Create and process daily bars request
let request = 
    new DailyBarsRequest(
        Identifier = new Identifier( Exchange = exchange, Ticker = ticker)
    )
let reply = service.DailyBars(request, headers)

The following code snippet displays the *daily bars* data points:

In [ ]:
reply.Data
|> display

### Step 4: Visualize data

#### 4.1 Plot the closing price and volume
The following code snippets retrive *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package. Plot a simple line graph with the *close price* and *volume* to easily spot spikes in trading as follows: 

In [ ]:
// Extract the close price and the volume
let closePrices = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Close))
let volumes = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume))

In [ ]:
// Create the graph
let closesGraph = 
    seq {
        new Scattergl(
            name = "Close Price",
            mode = "lines",
            x = (closePrices |> Seq.map fst),
            y = (closePrices |> Seq.map snd)
        )
        new Scattergl(
            name = "Volume",
            mode = "bar",
            x = (volumes |> Seq.map fst),
            y = (volumes |> Seq.map snd),
            yaxis = "y2"
        )
    }


let layout = 
    new Layout.Layout(
        title = $"Close price & Volume for {ticker}",
        yaxis = new Yaxis(
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Volume", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(closesGraph, layout)
|> Chart.WithXTitle "Time"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

#### 4.2 Plot the bars chart
To generate the bars graph we need to pass Open, High, Low and Close prices as per parameters:

In [ ]:
let bars = 
    reply.Data
    |> Seq.map ( fun b -> (new DateTime(b.Date.Year, b.Date.Month, b.Date.Day), b.Open, b.High, b.Low, b.Close))

In [ ]:
bars
|> Chart.Candlestick
|> Chart.WithTitle $"Bars chart for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "OHLC"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display